In [13]:
%pip install llama-index-embeddings-openai
%pip install llama-index-readers-file
%pip install llama-index-llms-openai
%pip install camelot-py
%pip install opencv-python
%pip install ghostscript

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [6]:
import camelot # extract text-based tables from PDFs

# https://en.wikipedia.org/wiki/The_World%27s_Billionaires
from llama_index.core import VectorStoreIndex
from llama_index.core.query_engine import PandasQueryEngine
from llama_index.core.schema import IndexNode
from llama_index.llms.openai import OpenAI

from llama_index.readers.file import PyMuPDFReader
from typing import List

# Default Settings

In [8]:
import os
# os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

In [9]:
file_path = "files/billionaires_page.pdf"

# initialize PDF reader
reader = PyMuPDFReader()
docs = reader.load(file_path)


In [10]:
# use camelot to parse tables
def get_tables(path: str, pages: List[int]):
    table_dfs = []
    for page in pages:
        table_list = camelot.read_pdf(path, pages=str(page))
        table_df = table_list[0].df
        table_df = (
            table_df.rename(columns=table_df.iloc[0])
            .drop(table_df.index[0])
            .reset_index(drop=True)
        )
        table_dfs.append(table_df)
    return table_dfs

In [22]:
# Read page 4 AND 25
table_dfs = get_tables(file_path, pages=[4, 38]) 

# shows list of top billionaires in 2023
table_dfs[0]

,No.,Name,Net worth\n(USD),Age,Nationality,Primary\nsource(s) of\nwealth
0,1,Bernard\nArnault &\nfamily,$211 billion,74,France,LVMH
1,2,Elon Musk,$180 billion,51,United States/\n South Africa,"Tesla, SpaceX"
2,3,Jeff Bezos,$114 billion,59,United States,Amazon
3,4,Larry Ellison,$107 billion,78,United States,Oracle Corporation
4,5,Warren Buffett,$106 billion,92,United States,Berkshire\nHathaway
5,6,Bill Gates,$104 billion,67,United States,Microsoft
6,7,Michael\nBloomberg,$94.5 billion,81,United States,Bloomberg L.P.
7,8,Carlos Slim &\nfamily,$93 billion,83,Mexico,"Telmex, América\nMóvil, Grupo\nCarso"
8,9,Mukesh\nAmbani,$83.4 billion,65,India,Reliance\nIndustries
9,10,Steve Ballmer,$80.7 billion,67,United States,Microsoft


In [23]:
# shows list of top billionaires
table_dfs[1]

,Year,Number of billionaires,Group's combined net worth
0,2023[2],"2,640",$12.2 trillion
1,2022[6],"2,668",$12.7 trillion
2,2021[11],"2,755",$13.1 trillion
3,2020,"2,095",$8.0 trillion
4,2019,"2,153",$8.7 trillion
5,2018,"2,208",$9.1 trillion
6,2017,"2,043",$7.7 trillion
7,2016,"1,810",$6.5 trillion
8,2015[18],"1,826",$7.1 trillion
9,2014[67],"1,645",$6.4 trillion


# Create Pandas Query Engines

In [24]:
# define query engines over these tables
llm = OpenAI(model="gpt-4")

df_query_engines = [
    PandasQueryEngine(table_df, llm=llm) for table_df in table_dfs
]

## Q1: What's the net worth of the second richest billionaire in 2023?

In [25]:
response = df_query_engines[0].query(
    "What's the net worth of the second richest billionaire in 2023?"
)
print(str(response))

$180 billion


In [27]:
# Elon with $180 billion!
table_dfs[0].head(2)

,No.,Name,Net worth\n(USD),Age,Nationality,Primary\nsource(s) of\nwealth
0,1,Bernard\nArnault &\nfamily,$211 billion,74,France,LVMH
1,2,Elon Musk,$180 billion,51,United States/\n South Africa,"Tesla, SpaceX"


In [28]:
## Q2: How many billionaires were there in 2009?
response = df_query_engines[1].query(
    "How many billionaires were there in 2009?"
)
print(str(response))

793


In [32]:
# 2.4 trillion!
table_dfs[1].iloc[[14]]

,Year,Number of billionaires,Group's combined net worth
14,2009,793,$2.4 trillion


In [43]:
try:
    response = df_query_engines[0].query(
        "Name the billionaires from fashion"
    )
    print(str(response))
except:
    print("Error")

0    Bernard\nArnault &\nfamily
Name: Name, dtype: object


In [44]:
try:
    response = df_query_engines[0].query(
        "Describe what is in the table"
    )
    print(str(response))
except:
    print("Error")

       No.                        Name Net worth\n(USD) Age    Nationality  \
count   10                          10               10  10             10   
unique  10                          10               10   9              5   
top      1  Bernard\nArnault &\nfamily     $211 billion  67  United States   
freq     1                           1                1   2              6   

       Primary\nsource(s) of\nwealth  
count                             10  
unique                             9  
top                        Microsoft  
freq                               2  


In [46]:
try:
    response = df_query_engines[0].query(
        "Name the oldest billionaire and show the age as well"
    )
    print(str(response))
except:
    print("Error")

Name    Warren Buffett
Age                 92
Name: 4, dtype: object


In [47]:
try:
    response = df_query_engines[0].query(
        "Which year is this data from?"
    )
    print(str(response))
except:
    print("Error")

There was an error running the output as Python code. Error message: invalid syntax (<unknown>, line 1)


Traceback (most recent call last):
  File "/Users/shane/Documents/Github/ai-exploration/.venv/lib/python3.11/site-packages/llama_index/core/query_engine/pandas/output_parser.py", line 39, in default_output_processor
    tree = ast.parse(output)
           ^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/ast.py", line 50, in parse
    return compile(source, filename, mode, flags,
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<unknown>", line 1
    The data provided does not contain any information about the year.
        ^^^^
SyntaxError: invalid syntax


In [48]:
try:
    response = df_query_engines[0].query(
        "What is the combined networth from the top three billionaires?"
    )
    print(str(response))
except:
    print("Error")

There was an error running the output as Python code. Error message: could not convert string to float: '211\xa0billion'


Traceback (most recent call last):
  File "/Users/shane/Documents/Github/ai-exploration/.venv/lib/python3.11/site-packages/llama_index/core/query_engine/pandas/output_parser.py", line 53, in default_output_processor
    output_str = str(safe_eval(module_end_str, {"np": np}, local_vars))
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/shane/Documents/Github/ai-exploration/.venv/lib/python3.11/site-packages/llama_index/core/exec_utils.py", line 140, in safe_eval
    return eval(__source, _get_restricted_globals(__globals), __locals)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 1, in <module>
  File "/Users/shane/Documents/Github/ai-exploration/.venv/lib/python3.11/site-packages/pandas/core/generic.py", line 6534, in astype
    new_data = self._mgr.astype(dtype=dtype, copy=copy, errors=errors)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/shane/Documents/Github/

In [ ]:
# Notes: Proper data cleaning is required. E.g convert strings to float

# Build Vector Index
Build vector index over the chunked document as well as over the additional IndexNode objects linked to the tables.

In [49]:
from llama_index.core import Settings

doc_nodes = Settings.node_parser.get_nodes_from_documents(docs)

In [51]:
# Each page is one document. This document continas 49 pages.
print(len(docs))
docs

49


[Document(id_='e3abcc44-d104-458f-a623-1d5c9301392f', embedding=None, metadata={'total_pages': 49, 'file_path': 'files/billionaires_page.pdf', 'source': '1'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text="22/02/2024, 21:33\nThe World's Billionaires - Wikipedia\nhttps://en.wikipedia.org/wiki/The_World%27s_Billionaires\n1/49\nThe World's Billionaires\nList of the world's billionaires, ranked in order of net worth\nThe net worth of the world's billionaires increased from\nless than US$1 trillion in 2000 to over $7 trillion in 2015.\nPublication details\nPublisher\nWhale Media Investments\nForbes family\nPublication\nForbes\nFirst published\nMarch 1987[1]\nLatest publication\nApril 4, 2023\nCurrent list details (2023)[2]\nWealthiest\nBernard Arnault\nNet worth (1st)\n\xa0US$211\xa0billion\nNumber of\nbillionaires\n\xa02,640 (from 2668)\nTotal list net worth\nvalue\n\xa0US$12.2\xa0trillion (from US$12.7\ntrillion)\nNumber of women\n\xa0337\nNumber o

In [52]:
# define index nodes
summaries = [
    (
        "This node provides information about the world's richest billionaires"
        " in 2023"
    ),
    (
        "This node provides information on the number of billionaires and"
        " their combined net worth from 2000 to 2023."
    ),
]

df_nodes = [
    IndexNode(text=summary, index_id=f"pandas{idx}")
    for idx, summary in enumerate(summaries)
]

df_id_query_engine_mapping = {
    f"pandas{idx}": df_query_engine
    for idx, df_query_engine in enumerate(df_query_engines)
}

In [54]:
doc_nodes

[TextNode(id_='bd23fe50-ac63-4bf2-86aa-27aac7161434', embedding=None, metadata={'total_pages': 49, 'file_path': 'files/billionaires_page.pdf', 'source': '1'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='e3abcc44-d104-458f-a623-1d5c9301392f', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'total_pages': 49, 'file_path': 'files/billionaires_page.pdf', 'source': '1'}, hash='5c55bb0103eb25e6f0a0eb120254f158109a95c5c92d0103061876b6cb0db482'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='77bfa638-07d5-4e56-a330-2fa64d6f82ce', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='9c43bad2eac0b91ead3e50c39fa680f32407d7856049839d73c5b6ba63a2afd5')}, text="22/02/2024, 21:33\nThe World's Billionaires - Wikipedia\nhttps://en.wikipedia.org/wiki/The_World%27s_Billionaires\n1/49\nThe World's Billionaires\nList of the world's billionaires, ranked in order of net worth\nThe net worth of the world's bil

In [55]:
df_nodes

[IndexNode(id_='3e0edee5-31c2-4722-83cd-a35aba8d8f44', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text="This node provides information about the world's richest billionaires in 2023", start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n', index_id='pandas0', obj=None),
 IndexNode(id_='b9bb75b1-a9e2-4bd0-ab08-c39d51f6a2d5', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='This node provides information on the number of billionaires and their combined net worth from 2000 to 2023.', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n', index_id='pandas1', obj=None)]

In [68]:
# construct top-level vector index + query engine
vector_index = VectorStoreIndex(doc_nodes + df_nodes)
vector_retriever = vector_index.as_retriever(similarity_top_k=3)

# Use RecursiveRetriever in our RetrieverQueryEngine
## NOT WORKING
We define a RecursiveRetriever object to recursively retrieve/query nodes. We then put this in our RetrieverQueryEngine along with a ResponseSynthesizer to synthesize a response.

We pass in mappings from id to retriever and id to query engine. We then pass in a root id representing the retriever we query first.

In [69]:
# baseline vector index (that doesn't include the extra df nodes).
# used to benchmark
vector_index0 = VectorStoreIndex(doc_nodes)
vector_query_engine0 = vector_index0.as_query_engine()

In [70]:
from llama_index.core.retrievers import RecursiveRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import get_response_synthesizer

recursive_retriever = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever},
    query_engine_dict=df_id_query_engine_mapping,
    verbose=True,
)

response_synthesizer = get_response_synthesizer(response_mode="compact")

query_engine = RetrieverQueryEngine.from_args(
    recursive_retriever, response_synthesizer=response_synthesizer
)

In [71]:
response = query_engine.query(
    "What's the net worth of the second richest billionaire in 2023?"
)

Retrieving with query id None: What's the net worth of the second richest billionaire in 2023?
Retrieving text node: 22/02/2024, 21:33
The World's Billionaires - Wikipedia
https://en.wikipedia.org/wiki/The_World%27s_Billionaires
3/49
The rankings are published annually in March, so the net worths listed are
snapshots taken at that time. These lists only show the top 10 wealthiest
billionaires for each year.
Icon
Description
Has not changed from the previous ranking.
Has increased from the previous ranking.
Has decreased from the previous ranking.
In the 37th annual Forbes list of the world's billionaires, the list included 2,640
billionaires with a total net wealth of $12.2 trillion, down 28 members and $500
billion from 2022. Over half of the list is less wealthy compared to the previous
year, including Elon Musk, who fell from No. 1 to No. 2.[2] The list also marks for
the first time a French citizen was in the top position as well as a non-American
for the first time since 2013 when

In [72]:
response = query_engine.query(
    "What's the net worth of the second richest billionaire in 2023? Please only pride the net worth amount in the answer"
)

Retrieving with query id None: What's the net worth of the second richest billionaire in 2023? Please only pride the net worth amount in the answer
Retrieving text node: 22/02/2024, 21:33
The World's Billionaires - Wikipedia
https://en.wikipedia.org/wiki/The_World%27s_Billionaires
3/49
The rankings are published annually in March, so the net worths listed are
snapshots taken at that time. These lists only show the top 10 wealthiest
billionaires for each year.
Icon
Description
Has not changed from the previous ranking.
Has increased from the previous ranking.
Has decreased from the previous ranking.
In the 37th annual Forbes list of the world's billionaires, the list included 2,640
billionaires with a total net wealth of $12.2 trillion, down 28 members and $500
billion from 2022. Over half of the list is less wealthy compared to the previous
year, including Elon Musk, who fell from No. 1 to No. 2.[2] The list also marks for
the first time a French citizen was in the top position as well

In [73]:
response.source_nodes[0].node.get_content()

"22/02/2024, 21:33\nThe World's Billionaires - Wikipedia\nhttps://en.wikipedia.org/wiki/The_World%27s_Billionaires\n3/49\nThe rankings are published annually in March, so the net worths listed are\nsnapshots taken at that time. These lists only show the top 10 wealthiest\nbillionaires for each year.\nIcon\nDescription\nHas not changed from the previous ranking.\nHas increased from the previous ranking.\nHas decreased from the previous ranking.\nIn the 37th annual Forbes list of the world's billionaires, the list included 2,640\nbillionaires with a total net wealth of $12.2 trillion, down 28 members and $500\nbillion from 2022. Over half of the list is less wealthy compared to the previous\nyear, including Elon Musk, who fell from No. 1 to No. 2.[2] The list also marks for\nthe first time a French citizen was in the top position as well as a non-American\nfor the first time since 2013 when the Mexican Carlos Slim Helu was the world's\nrichest person. The list, like in 2022, counted 15 u

In [74]:
str(response)

'$96.5 billion'

In [76]:
response = query_engine.query("How many billionaires were there in 2009?")

Retrieving with query id None: How many billionaires were there in 2009?
Retrieving text node: 22/02/2024, 21:33
The World's Billionaires - Wikipedia
https://en.wikipedia.org/wiki/The_World%27s_Billionaires
19/49
The 2010 list featured 164 re-entries and 97 true newcomers.[8] Asia accounted
for more than 100 of the new entrants. Overall, just 12 percent of the list lost
wealth since 2009, and 30 people fell off the list. 13 others died.[39] Of the 89
women, 12 were newcomers in 2010. Steve Forbes said the growing number of
billionaires was a clear sign that the world's economy was recovering from 2009's
global financial crisis.[8]
In June 2010, Gates and Buffett announced the Giving Pledge, a promise to give
the majority of their wealth to philanthropic causes. As of 2017, the pledge had
158 signatories, but some of the signatories have since died. Most of the signers of
the pledge are billionaires, and their pledges total over $365 billion.[41]
No.
Name
Net worth
(USD)
Age
Nationality

In [77]:
str(response)

'1,011 people made the 2010 list of billionaires.'

In [66]:
response = vector_query_engine0.query(
    "How many billionaires were there in 2009?"
)

print(response.source_nodes[0].node.get_content())

22/02/2024, 21:33
The World's Billionaires - Wikipedia
https://en.wikipedia.org/wiki/The_World%27s_Billionaires
19/49
The 2010 list featured 164 re-entries and 97 true newcomers.[8] Asia accounted
for more than 100 of the new entrants. Overall, just 12 percent of the list lost
wealth since 2009, and 30 people fell off the list. 13 others died.[39] Of the 89
women, 12 were newcomers in 2010. Steve Forbes said the growing number of
billionaires was a clear sign that the world's economy was recovering from 2009's
global financial crisis.[8]
In June 2010, Gates and Buffett announced the Giving Pledge, a promise to give
the majority of their wealth to philanthropic causes. As of 2017, the pledge had
158 signatories, but some of the signatories have since died. Most of the signers of
the pledge are billionaires, and their pledges total over $365 billion.[41]
No.
Name
Net worth
(USD)
Age
Nationality
Source(s) of wealth
1 
Carlos Slim &
family
$53.5 billion 
70
 Mexico
América Móvil, Grupo
Car

In [67]:
print(str(response))

There were 10 billionaires in 2009.


# This works - using two tables with the associated metadata only

In [79]:
df_nodes

[IndexNode(id_='3e0edee5-31c2-4722-83cd-a35aba8d8f44', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text="This node provides information about the world's richest billionaires in 2023", start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n', index_id='pandas0', obj=None),
 IndexNode(id_='b9bb75b1-a9e2-4bd0-ab08-c39d51f6a2d5', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='This node provides information on the number of billionaires and their combined net worth from 2000 to 2023.', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n', index_id='pandas1', obj=None)]

In [78]:
# construct top-level vector index + query engine
vector_index = VectorStoreIndex(df_nodes)
vector_retriever = vector_index.as_retriever(similarity_top_k=1)

In [84]:
from llama_index.core.retrievers import RecursiveRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import get_response_synthesizer

recursive_retriever = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever},
    query_engine_dict=df_id_query_engine_mapping,
    verbose=True,
)

response_synthesizer = get_response_synthesizer(response_mode="compact")

query_engine = RetrieverQueryEngine.from_args(
    recursive_retriever, response_synthesizer=response_synthesizer
)

In [86]:
response = query_engine.query(
    "What's the net worth of the second richest billionaire in 2023?"
)

Retrieving with query id None: What's the net worth of the second richest billionaire in 2023?
Retrieved node with id, entering: pandas0
Retrieving with query id pandas0: What's the net worth of the second richest billionaire in 2023?
Got response: $180 billion


In [87]:
response = query_engine.query(
    "How many billionaires were there in 2009?"
)

Retrieving with query id None: How many billionaires were there in 2009?
Retrieved node with id, entering: pandas1
Retrieving with query id pandas1: How many billionaires were there in 2009?
Got response: 793
